In [1]:
import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import sys
sys.path.append("../../")
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.modeltcan import TCAN
import numpy as np
import argparse

In [3]:
def customOneHotEncoder(data):
    dataAdjust = data.ljust(200,'0')[:200] # padding if not of length and adjusting the data lenght to get a 200x39 input matrix
    # define universe of possible input values
    alphabet = '0123456789abcdefghijklmnopqrstuvwxyz,._'
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    # integer encode input data
    integer_encoded = [char_to_int[char] for char in dataAdjust]
    #print(integer_encoded)
    # one hot encode
    onehot_encoded = list()
    for i, value in enumerate(integer_encoded):
        letter = [0 for _ in range(len(alphabet))]
        letter[value] = 1
        onehot_encoded.append(letter)
    #print(onehot_encoded) # the real encoding
    return onehot_encoded

# takes a .csv filename
def dataPreprocessing(fileName):
    df = pd.read_csv(fileName, header = None)
    
    #prepare the imput data
    xString = df.iloc[:,:41].to_string(header=False, index=False, index_names = False).split('\n')
    xList = [','.join(ele.split()) for ele in xString] # gives comma separated strings for each row of DataFrame
    xData = []
    for string in xList:
        stringLower = string.lower()
        oneHot = customOneHotEncoder(stringLower)
        xData.append(oneHot)
    xMid = np.array(xData)
    xArray = xMid.transpose(0,2,1) # convert xMid's dim (size, 200, 39) to (size, 39, 200)
    
    #prepare the label data
    df[41] = np.where(df[41]=='normal', 'normal', 'attack') # replacing anything except 'normal' with 'attack'
    Ydf = df[41]
    #labelName = Ydf.unique().tolist().sort() # sorted 38 label names
    #yArray = Ydf.str.get_dummies().to_numpy() # ndarray of shape(rows/lines, 38)
    yArray = Ydf.to_numpy()
    
    assert xArray.shape[0] == yArray.shape[0], 'unequal input and label sample size'
    
    
    return xArray, yArray # return processed array of input and label

In [4]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

class NSLKDDDataset(Dataset):
    def __init__(self, fileName):
        self.data = pd.read_csv(fileName, header = None)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # prepare x data
        string = list(','.join('%s' %x for x in y) for y in self.data.iloc[[idx], :41].values)
        stringLower = string[0].lower()
        xData = customOneHotEncoder(stringLower) # Dim (200, 39)
        xMid = np.array(xData)
        xArray = xMid.transpose() # should be now (39, 200)
        
        # prepate y data
        #self.data.iloc[idx, 41] = np.where(self.data.iloc[idx, 41]=='normal', 0, 1) # replacing normals with 0 and anything else with 1
        yArray = np.where(self.data.iloc[idx, 41]=='normal', 0, 1)
        
        #yArray = Ydf.to_numpy()
    
        #assert xArray.shape == yArray.shape, 'unequal input and label sample size'
        
        return torch.from_numpy(xArray), torch.from_numpy(yArray) # returns torch tensor of x and y

In [5]:
params = {'batch_size': 64, 'shuffle': True}
fileNameTrain = 'KDDTrain+.csv'
fileNameTest = 'KDDTest+.csv'
datasetTrain = NSLKDDDataset(fileNameTrain)
datasetTest = NSLKDDDataset(fileNameTest)
dataGeneratorTrain = DataLoader(datasetTrain, **params)
dataGeneratorTest = DataLoader(datasetTest, **params)


In [6]:
root = './data/mnist'
batch_size = 64
n_classes = 2
input_channels = 39
seq_length = int(200)
epochs = 100
steps = 0

In [21]:
#train_loader, test_loader = data_generator(root, batch_size)

permute = torch.Tensor(np.random.permutation(784).astype(np.float64)).long()
channel_sizes = [32] * 6 #hidden nodes times levels 
kernel_size = 5
nheads = 1

model = TCAN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=0.25)
#input_size, output_size, num_channels, 
#kernel_size, dropout, num_sub_blocks, temp_attn, nheads, en_res,                              num_sub_blocks
#conv, key_size
#
# nheads, key_size


lr = 1e-5
optimizer = getattr(optim, 'Adam')(model.parameters(), lr=lr)

TypeError: __init__() missing 8 required positional arguments: 'num_channels', 'num_sub_blocks', 'temp_attn', 'nheads', 'en_res', 'conv', 'key_size', and 'visual'

In [7]:
def train(ep):
    global steps
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(dataGeneratorTrain):
        # print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        optimizer.zero_grad()
        data = data.view(-1, input_channels, seq_length)
        data, target = Variable(data), Variable(target)
        #print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        #print(target)
        optimizer.zero_grad()
        #print(data[0])
        data = data.type(torch.FloatTensor)
        output = model(data)
        #print(output.shape)
        target = target.type(torch.LongTensor)
        #loss1 = torch.nn.CrossEntropyLoss()
        loss = F.nll_loss(output, target) # negative log likelihood
        #loss = loss1(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss
        steps += seq_length
        if batch_idx > 0 and batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * batch_size, len(dataGeneratorTrain.dataset),
                100. * batch_idx / len(dataGeneratorTrain), train_loss.item()/100, steps))
            train_loss = 0

In [8]:
def test():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorTest:
            model.eval()
            #data = data.view(-1, input_channels, seq_length)
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            #loss1 = torch.nn.CrossEntropyLoss()
            test_loss += F.nll_loss(output, target, size_average=False).item()
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(dataGeneratorTest.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorTest.dataset),
            100. * correct / len(dataGeneratorTest.dataset)))
        return test_loss

In [ ]:
if __name__ == "__main__":
    for epoch in range(1, epochs+1):
        train(epoch)
        test()
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

Train Epoch: 1 [6400/125973 (5%)]	Loss: 0.700333	Steps: 20200
Train Epoch: 1 [12800/125973 (10%)]	Loss: 0.692618	Steps: 40200
Train Epoch: 1 [19200/125973 (15%)]	Loss: 0.691375	Steps: 60200
Train Epoch: 1 [25600/125973 (20%)]	Loss: 0.691529	Steps: 80200
Train Epoch: 1 [32000/125973 (25%)]	Loss: 0.690802	Steps: 100200
Train Epoch: 1 [38400/125973 (30%)]	Loss: 0.691262	Steps: 120200
Train Epoch: 1 [44800/125973 (36%)]	Loss: 0.691608	Steps: 140200
Train Epoch: 1 [51200/125973 (41%)]	Loss: 0.689336	Steps: 160200
Train Epoch: 1 [57600/125973 (46%)]	Loss: 0.689111	Steps: 180200
Train Epoch: 1 [64000/125973 (51%)]	Loss: 0.690009	Steps: 200200
Train Epoch: 1 [70400/125973 (56%)]	Loss: 0.688937	Steps: 220200
Train Epoch: 1 [76800/125973 (61%)]	Loss: 0.689055	Steps: 240200
Train Epoch: 1 [83200/125973 (66%)]	Loss: 0.688812	Steps: 260200
Train Epoch: 1 [89600/125973 (71%)]	Loss: 0.689421	Steps: 280200
Train Epoch: 1 [96000/125973 (76%)]	Loss: 0.687710	Steps: 300200
Train Epoch: 1 [102400/125973 (

C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.6951, Accuracy: 9711/22544 (43%)

Train Epoch: 2 [6400/125973 (5%)]	Loss: 0.686933	Steps: 414000
Train Epoch: 2 [12800/125973 (10%)]	Loss: 0.678061	Steps: 434000
Train Epoch: 2 [19200/125973 (15%)]	Loss: 0.673154	Steps: 454000
Train Epoch: 2 [25600/125973 (20%)]	Loss: 0.666039	Steps: 474000
Train Epoch: 2 [32000/125973 (25%)]	Loss: 0.657392	Steps: 494000
Train Epoch: 2 [38400/125973 (30%)]	Loss: 0.646876	Steps: 514000
Train Epoch: 2 [44800/125973 (36%)]	Loss: 0.632886	Steps: 534000
Train Epoch: 2 [51200/125973 (41%)]	Loss: 0.616019	Steps: 554000
Train Epoch: 2 [57600/125973 (46%)]	Loss: 0.586588	Steps: 574000
Train Epoch: 2 [64000/125973 (51%)]	Loss: 0.553573	Steps: 594000
Train Epoch: 2 [70400/125973 (56%)]	Loss: 0.511505	Steps: 614000
Train Epoch: 2 [76800/125973 (61%)]	Loss: 0.457157	Steps: 634000
Train Epoch: 2 [83200/125973 (66%)]	Loss: 0.406290	Steps: 654000
Train Epoch: 2 [89600/125973 (71%)]	Loss: 0.357387	Steps: 674000
Train Epoch: 2 [96000/125973 (7

In [ ]:
from sklearn.metrics import classification_report
def get_metrics(dataLoader):
    total = 0
    correct = 0
    precision = 0
    recall = 0
    f1_score = 0
    accuracy = 0
    with torch.no_grad():
        for data, target in dataLoader: # just 1 batch
            model.eval()
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            #print(predicted)
            total+=target.size(0)
            correct+=(predicted == target).sum().item()
            report = classification_report(target, predicted, output_dict=True)
            precision += report['macro avg']['precision']
            recall += report['macro avg']['recall']
            f1_score += report['macro avg']['f1-score']
            accuracy += report['accuracy']
            print(report)
    print("Precision: {}, Recall: {}, F1-Score: {}, Accuracy: {}, AccuracyCust: {}".format(precision, recall, f1_score, accuracy, correct/total))
    

In [ ]:
fileNameVal = 'Ds.csv'
datasetVal = NSLKDDDataset(fileNameVal)
params = {'batch_size': 22544, 'shuffle': True}
dataGeneratorVal = DataLoader(datasetVal, **params)
#RuntimeError: expected scalar type Int but found Float
#get_metrics(dataGeneratorVal)
def val():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorVal:
            model.eval()
            #data = data.view(-1, input_channels, seq_length)
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            #loss1 = torch.nn.CrossEntropyLoss()
            #test_loss += F.nll_loss(output, target, size_average=False).item()
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            print(pred)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(dataGeneratorTest.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorVal.dataset),
            100. * correct / len(dataGeneratorVal.dataset)))
        return test_loss

val()


In [ ]:
get_metrics(dataGeneratorVal)

In [ ]:
get_metrics(dataGeneratorTrain)